### Company Matching

Many use cases of Factiva Analytics involve tying Factiva content to individual companies. Dow Jones enhances our Factiva content with several intelligent identifiers across subject, region, or industry. 

Companies are tagged in multiple ways. The two most important distinctions are seen in the `company_codes_about` and `company_codes_occur` columns. These determine whether an article is "about" a company, or whether a company is merely mentioned in the article.

Once you know the company Factiva codes (or `fcodes`,) you can easily locate articles at either threshold.

### Public Company Matching

Public companies are very straightforward to match using our Factiva Analytics APIs. Taxonomy endpoints can retrieve tables of all company Dow Jones company fcodes.

Factiva Analytics comes with a taxonomy endpoint that allows mapping files to be downloaded. All Factiva Analytics customers have access to `ticker` and `exchange` information. Talk to your account rep if you would like to receive acces to `ISIN`, `CUSIP`, and `SEDOL` identifiers. If your company contracts with these providers, we'd be able to turn on access for you.

Our Factiva News Python library wraps these endpoints in a way that is easy to use.

**Note**: Not all companies will include these identifiers. Many fcodes will only have company descriptions.

**Note**: This company mapping file contains ~2M companies. Factiva Analytics only newscodes ~400K companies. Just because an fcode is showing up in this company mapping file does not mean that it will be tagged in our database.

More information can be found in our [developer portal](https://developer.dowjones.com/site/docs/factiva_apis/factiva_analytics_apis/factiva_snapshots_api/index.gsp) under "Taxonomy Lookup Requests."

More information about the Factiva News Python library can be found in its [Github Repo](https://github.com/dowjones/factiva-news-python).

##### Example

The below example shows how to map companies in the S&P 500.

In [1]:
import requests
import os
from zipfile import ZipFile
import xml.etree.ElementTree as etree

import pandas as pd

from thefuzz import fuzz

from factiva.core import UserKey
from factiva.news import Taxonomy



key_headers = {
    'user-key': os.environ['snapshot_API_KEY'], # Enter your API key here
    'content-type': "application/json",
    'cache-control': "no-cache"
}

Lets say we want to look up fcodes of companies in the S&P 500 by ticker. 

In [2]:
sp500 = pd.read_csv('./data/company-data/sp500.csv')
sp500.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


We could use the Factiva News Python library to map these companies:

In [3]:
u = UserKey(key=os.environ['snapshot_API_KEY'])
t = Taxonomy(user_key=u)

mapped_companies = t.get_company('ticker', sp500['Symbol'].tolist())
mapped_companies.head()

,id,fcode,common_name
0,ALGN,ALIGNT,Align Technology Inc.
1,DE,DEERE,Deere & Co.
2,EL,LAUDR,Estee Lauder Cos. Cl A
3,WHR,WHRLPL,Whirlpool Corp.
4,MS,DWITD,Morgan Stanley


A full taxonomy mapping file can be downloaded directly from the endpoint, if you'd prefer to map companies manually:

In [4]:
taxonomy_url = 'https://api.dowjones.com/alpha/taxonomies/companies/csv'

file_resp = requests.get(taxonomy_url, headers=key_headers, allow_redirects=True, stream=True)

filename = 'companies.csv'

filesize = file_resp.headers['Content-length']
with open(filename, 'wb') as local_file:
    for chunk in file_resp.iter_content(chunk_size=1000000000):
        local_file.write(chunk)

company_taxonomy = pd.read_csv('companies.csv')

**Note**: There may be multiple tickers per exchange. I am limiting this to popular exchanges for the S&P 500.

In [5]:
popular_exchanges = company_taxonomy[company_taxonomy[' primary_exchange'].isin(['XNYS', 'XNAS', 'XASX', 'BATS'])]

popular_exchanges = popular_exchanges.drop_duplicates(subset=[' ticker'])

We could just simply map the tickers we have with the mapping file

In [6]:
merged = pd.merge(sp500, popular_exchanges, left_on='Symbol', right_on=' ticker', how='inner')
merged = merged[['Symbol', 'code', ' description']]
merged.columns = ['Ticker', 'FCode', 'Description']
merged.head()


,Ticker,FCode,Description
0,MMM,MMMUK,3M Company
1,AOS,AOSM,A.O. Smith Corp
2,ABT,ABOTLB,Abbott Laboratories Inc.
3,ABBV,ABVBBA,AbbVie Inc.
4,ABMD,ABMD,Abiomed Inc


### Matching companies by DUNS

We have one more identifier you are able to use to map companies. `DUNS` can be used to map many private companies you may have to `fcodes`.

`DUNS` matching is not traditionally part of the Factiva Analytics product. If you require to map companies using these codes, you will need to use our `Factiva Select Taxonomy Feed`. This is an XML feed that provides additional company information not found in the Factiva Analytics Taxonomy API.

`DUNS` are available for a wider variety of companies, including private companies.

More information can be found in our [developer portal](https://developer.dowjones.com/site/docs/factiva_feeds/workflows/factiva_taxonomy_feed/Company_Taxonomy_Product_Guidelines_v3_1.gsp)


##### Example

The below example shows how to map a selection of companies using `DUNS`

First we download the files

In [7]:
username = os.environ['taxonomy_feed_USERNAME']
password = os.environ['taxonomy_feed_PASSWORD']

def download_files(stub, base_url, username, password):
    # Downloads all files, returns file list
    filelist_url = base_url + 'filelist.asp?files=all'
    ftp_file_list = requests.get(filelist_url, auth=(username, password))
    
    filename = sorted([i for i in ftp_file_list.text.split(',') if stub in i])[-1]
    file_url = base_url + filename
    
    file_resp = requests.get(file_url, auth=(username, password))
    
    with open(filename, 'wb') as local_file:
        for chunk in file_resp.iter_content(chunk_size=1000000000):
            local_file.write(chunk)
            
    with ZipFile(filename, 'r') as zipObj:
        file_list = zipObj.namelist()
        zipObj.extractall()
        
    return file_list

duns_unlisted_file_list = download_files('f_$co07', 'https://select.factiva.com/Symbology_MAP/', username, password)
duns_listed_file_list = download_files('f_$co08', 'https://select.factiva.com/Symbology_MAP/', username, password)


Then parse the XML files

In [8]:
duns_record_list = []

for file in duns_unlisted_file_list + duns_listed_file_list:
    tree = etree.parse(file)
    symlist = tree.getroot()
    for symdoc in symlist.iter('SymDoc'):
        
        docdata = symdoc.find('Metadata').find('DocData')
        code_fields = symdoc.find('ReplyItem').find('Company')
        
        fcode = code_fields.get("fcode").lower()
        
        duns = code_fields.find('Code').get('value')
        if type(duns) != str:
            print(type(duns))
            
        
        row = {
            'company_code': fcode,
            'DUNS': duns
        }
        
        duns_record_list.append(row)
        
duns = pd.DataFrame(duns_record_list, dtype=str)


Lets read in a list of 5,000 companies and see if we can match them.

In [9]:
large_company_list = pd.read_csv('./data/company-data/large-company-list.csv', dtype=str)
large_company_list.head()

,Descriptor,Aliases,DUNS
0,"""Der Gruene Punkt "" – Duales System Deutschlan...",Dual System Deutschland AG|Duales System Deuts...,506924617
1,.com Solutions Inc,com Solutions Inc,106893055
2,22. decembar a.d. Banja Luka,22. decembar ad,643403780
3,26 Joint Stock Company,26 Joint Stock Company|26 JSC,555324536
4,27 Engineering Company Limited,NaN,NaN


In [10]:
duns_matched = pd.merge(large_company_list, duns, how='left', on='DUNS')
duns_matched.head()

,Descriptor,Aliases,DUNS,company_code
0,"""Der Gruene Punkt "" – Duales System Deutschlan...",Dual System Deutschland AG|Duales System Deuts...,506924617,dsydeu
1,.com Solutions Inc,com Solutions Inc,106893055,dotcms
2,22. decembar a.d. Banja Luka,22. decembar ad,643403780,ttdecm
3,26 Joint Stock Company,26 Joint Stock Company|26 JSC,555324536,tsxjsc
4,27 Engineering Company Limited,NaN,NaN,NaN


### How to deal with missing companies

You may notice that some of the `DUNS` in the large company list aren't filled in. It appears there are some companies for which we're not able to match using this method.

Lets take a look at what percentage of companies aren't matched:

In [11]:
duns_matched[~duns_matched['company_code'].isna()].shape[0] / duns_matched.shape[0]

0.8

Looks like we have 80% of our companies matched.

Just because we don't have a `DUNS` doesn't mean there isn't an fcode. There are a few choices you can make here.

If you have a tolerance for missing companies, you may want to stop here and accept a lower match rate. It may be possible to match more companies, but it may be more important to standardize your process and only work based on companies with codes.

If you don't have such tolerance, we will have to find another method to match these companies.

In [12]:
duns_matched[duns_matched['company_code'].isna()].head()

,Descriptor,Aliases,DUNS,company_code
4,27 Engineering Company Limited,NaN,NaN,NaN
11,4 pipes GmbH,NaN,NaN,NaN
12,407 ETR Concession Company Limited,NaN,NaN,NaN
18,8 Stars Contracting Co. W.L.L,NaN,NaN,NaN
24,A. J. Padelford & Son Inc.,NaN,NaN,NaN


Lets try to match some of these companies by name:

In [13]:
name_matched = pd.merge(duns_matched[duns_matched['company_code'].isna()], 
         company_taxonomy, how='left', left_on='Descriptor', right_on=' description')

name_matched = name_matched.drop_duplicates(subset='Descriptor')

In [14]:
name_matched[['Descriptor',  'code']].head()

,Descriptor,code
0,27 Engineering Company Limited,TSEREC
1,4 pipes GmbH,FOUPIG
2,407 ETR Concession Company Limited,FZSECC
3,8 Stars Contracting Co. W.L.L,EISCCW
4,A. J. Padelford & Son Inc.,AJPASI


Looks like we got more matches. Lets see if we're still missing any companies:

In [15]:
name_matched[name_matched['code'].isna()]

,Descriptor,Aliases,DUNS,company_code,code,description,cusip,isin,ticker,ticker_exchange,exchange,primary_exchange
147,Caves Bernard Massard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looks like we have one more company that doesn't match. If we wanted to aim for 100%, we'd need to try some more advanced matches. Below, as a last ditch and inefficient effort, we could try a fuzzy match:

In [20]:
match_company = 'Caves Bernard Massard'

company_taxonomy['fuzzy match'] = company_taxonomy[' description'].apply(lambda x: fuzz.ratio(match_company, x))

In [22]:
company_taxonomy.sort_values('fuzzy match', ascending=False)[['code', ' description', 'fuzzy match']].head()

,code,description,fuzzy match
73127,CBMLS,Caves Bernard-Massard,95
1271865,CBERNL,Charles Bernard Ltd.,68
668606,HARVEB,Harve Bernard,65
1826708,HBENAL,Harve Benard Ltd.,63
38454,BERSAS,Bernard Sas,62


### What if you still have missing companies?

Despite all this, some companies may not have a corresponding fcode. Unlike other company information providers, fcodes are only assigned to companies deemed `newsworthy`. In order to be newsworthy, a company will need to be regularly featured within our Factiva sources. 

Smaller companies may appear very sporadically in Factiva, but you may still be interested in being alerted when they do. In cases like this, you may decide to search for the company manually, using a keyword search.

You can read more about keyword searching in [Notebook 2.1: building large queries](https://github.com/dowjones/factiva-sample-notebooks/blob/47068eda143afde2cbe00fddd33280effc3d0362/2.1_complex_large_queries.ipynb). 

If you wanted to search for three companies: `CompanyA`, `CompanyB`, and `CompanyC`, you could perform a full text keyword search using the format below:

`REGEXP_CONTAINS(CONCAT(title, ' ', IFNULL(snippet, ''), ' ', IFNULL(body, '')), r'(?i)(^|\b)(CompanyA|CompanyB|CompanyC)($|.|\b)')`